In [34]:
#####
#Note
#####
#There is more functionality in the draft version - this is just a simple working version that's organised

#Uncomment to install the following packages
#%pip install googletrans==4.0.0-rc1 
#%pip install unidecode
#%pip install langdetect
#%pip install regex

#APIs and Libraries used for transaltion
from googletrans import Translator
from langdetect import detect
from unidecode import unidecode

#Imports
import os
import time
import regex

#Initialised translator
translator = Translator()
translator.raise_Exception = True

def runScript():
    path = "C:\\Users\\Rayha\\Desktop\\Voice Dramas - Copy"  #getDirPath()
    logPath = path.rsplit("\\", 1)[0] + "\\EngToAsiLangTransLog.txt"
    trasnlateFilesInDirectoy(path, logPath)

def trasnlateFilesInDirectoy(path, logPath):
    fullFilePaths, rootOfPaths, nameOfFiles = listFilePaths(path)
    translatedFileNames = translateList(nameOfFiles, logPath) #The 2nd arguements specifies where to write the text files showing all the before and after translations
    translatedFileNames = resolveDuplicates(translatedFileNames)

    i = 0
    for fullP, rootP, tranlatedFN in zip(fullFilePaths, rootOfPaths, translatedFileNames):
        os.rename(fullP, 
            os.path.join(rootP, (tranlatedFN + ".mp3"))
                .replace("\"", "'")
                .replace("?", "？")
                .replace(":", "﹕")
                .replace("\\", "⧵")
                .replace("/", "∕")
                .replace("*", "✲")
                .replace("<", "〈")
                .replace(">", "〉")
                .replace("|", "｜")
                .replace("‘", "'")
                [:250]
        )

        #To print progress
        print("[" + str(i) + "/" + str(len(fullFilePaths)) + "]: " + str(os.path.join(rootP, 
            (tranlatedFN + ".mp3"))
                .replace("\"", "'")
                .replace("?", "？")
                .replace(":", "﹕")
                .replace("\\", "⧵")
                .replace("/", "∕")
                .replace("*", "✲")
                .replace("<", "〈")
                .replace(">", "〉")
                .replace("|", "｜")
                .replace("‘", "'")
                [:250]
            )
        )
        i += 1
    
    print("> Done")

#################
#HELPER FUNCTIONS
#################

#Gets and formats the path of the directory that contains the MP3 files (and sub directories with MP3 files) that we want to rename with translations
def getDirPath():
    #Asks the user for a path to do the operations on
    path = input("File Path: ")

    #If there is no backslash in the path, the bath is not in the windows user path and the path doesn't exist, a new path is requested
    while "\\" not in path and ":\\Users" not in path and os.path.isdir(path):
        print("File Path Invalid")
        path = input("Please Select A Different File Path: ")
    #Gets rid of trailing backslashes or any file names attached to the end of the path
    if path.split("\\")[-1] == "" or "." in path.split("\\")[-1]:
        path = path.rsplit("\\", 1)[0]

#Gets, stores and returns the a list of filenames, parent directories and full paths or all the files in the given path and it's sub directories 
def listFilePaths(path):
    #These hold the filenames and the directories they're contained within seperarely and together
    fullFilePaths = [] 
    rootOfPaths = [] 
    nameOfFiles = []

    #Recursively goes through all directories and finds files within them
    for root, directories, files in os.walk(path):
        for file in files:
            fullFilePaths.append(os.path.join(root, file)) #The full path for each file is stored here
            nameOfFiles.append(file.rsplit(".", 1)[0]) #The names of each file found is stored here - the file extension is removes so it is not transalted
            rootOfPaths.append(root) #The path to each file's parent directory is stored her - this is prepended to the transalted filename later
    
    return fullFilePaths, rootOfPaths, nameOfFiles

#Translates list of filesnames
def translateList(untranslatedList, logPath):
    #Regular expression to gets ride "【...】" brackets and it's content if there is no hiragana, katakana or the word "drama" in them - optional!
    rePattern = regex.compile("【.*】", regex.UNICODE)
    hasHiraAndKataRE = r"^【+((.*[\p{Hiragana}\p{Katakana}]+.*)|(.*([Dd][Rr][Aa][Mm][Aa]).*))】+"

    #For primitive progress bar
    ###########################################
    #print("[", end="") 
    #noOfTrasnlationNeed = len(untranslatedList)
    #noOfTrasnlationsNeedForProgresTick = int(noOfTrasnlationNeed / 100)
    #progress = 0
    ###########################################

    #checks if the script needs to be resumed and loads the data needed to resume if it does
    untranslatedList_resumed, translatedFileNames_resumed, resumeIndex = resumeFromLastSession(logPath)
    if resumeIndex == -1:
        translatedFileNames = []
        translationNo = 1
        print(logBehaviour("##Start Of Logs##\n#################\n", logPath))
    else:
        untranslatedList = untranslatedList_resumed
        translatedFileNames = translatedFileNames_resumed
        translationNo = resumeIndex
    
    #Trasnlates filenames to english
    for i in range(translationNo -1, len(untranslatedList)):
        print(logBehaviour("\n> ", logPath).replace("\n", ""), end="")
        #time.sleep(0.5) #Puts the script to sleep inbetween translations to avoid HTTP 429 errors from sending too many requests at once
        
        #If filename is already fully in english, it is not traslated
        translation = untranslatedList[i]
        if isEnglish(untranslatedList[i]):
            translatedFileNames.append(translation)
        else:
            #Gets ride "【...】" brackets and it's content from the soon to be trasnlated text if there is no hiragana, katakana or the word "drama" in them them
            if regex.search(hasHiraAndKataRE, untranslatedList[i]) != None:
                print("IN! - " + untranslatedList[i])
                untranslatedList[i] = rePattern.sub("", untranslatedList[i])
                print("OUT! - " + untranslatedList[i])

            #A 429 error could occur if too many requests are made at once, so when that happens, the program is put to sleep (takes a short break) and then resumes
            try: 
                translation = translator.translate(untranslatedList[i][:250], dest="en").text[:250]
                translatedFileNames.append(translation)
            except:
                translatedFileNames = handle429Reponse(untranslatedList, translatedFileNames, untranslatedList[i], logPath, translationNo)
                translation = translatedFileNames[-1]
                continue

        print(logBehaviour("[" + str(translationNo) + "/" + str(len(untranslatedList)) + "]: " + untranslatedList[i] + "  \n\t\t===>  " + (translation if translation != untranslatedList[i] else "[[Same as before - no translation]]"), logPath))
        translationNo += 1

        #For primitive progress bar
        ##########################################
        #progress += 1
        #if (progress % noOfTrasnlationsNeedForProgresTick) == 0:
        #    print("#", end="") 
        #if progress == noOfTrasnlationNeed:
        #    print("]")
        ##########################################
    return translatedFileNames

#Checks for duplicates in the list of filename translations and resolves them by adding a suffix - which makes it unique
def resolveDuplicates(uncheckedList):
    duplicates = []
    for item in uncheckedList:
        if uncheckedList.count(item) > 1:
            #Makes note the duplicate
            duplicates.append(item)

    #Makes duplicates list distinct - removes duplicates from it
    duplicates = list(set(duplicates))

    #Handles duplicates if any are found
    if duplicates:
        #Goes through each duplicate name found and results all the duplicates with the name
        for dup in duplicates:
            #Character to within in the suffix "-[]" - which is appended to duplicates
            duplicateSuffixChar = "I"
            duplicateSuffix = "-[]"

            suffixRepeater = 1 #Keeps track of the amount of "I"s we want in the "-[]" suffix 

            #Gives each ducplidate item a suffix
            for i in range(len(uncheckedList)):
                if uncheckedList[i] == dup:
                    duplicateSuffix = duplicateSuffix[:2] + duplicateSuffixChar + duplicateSuffix[2:] #Adds 'I' within suffix before appending to the filename 
                    suffixRepeater += 1 #Insures the suffix will have an extra "I" when the next duplicate is found
                    uncheckedList[i] = uncheckedList[i] + duplicateSuffix #Adds suffix to translated file
                    
        #Checks if the duplicate handler introduced duplicates
        return resolveDuplicates(uncheckedList)
    else:
        return uncheckedList

#If the scipted ended abuptly in the last run, the log files will have data that this function can extract to allow the program to resume
def resumeFromLastSession(logPath):
    #Checks log file exists
    if os.path.exists(logPath):
        logFileContents = []

        #Reads contents of log files
        with open(logPath, encoding='utf8') as file:
            logFileContents = file.readlines()
            
        #Makes sure the log file has information to resume stored
        if len(logFileContents) >= 3:
            initialLogs = logFileContents[-2:] #Normal logs
            lastLogs = logFileContents[:-2] #Logs that indicate an abrupt and the state of the program during it

            if "##Script ended abruptly!" in lastLogs[0]:
                #Deletes log file to recreate them without the error logs at the end
                os.remove(logPath)
                with open(logPath, 'a+', encoding='utf8') as file: 
                    for lines in initialLogs:
                        file.write(lines + "\n")
                    file.close()

                return lastLogs[1].split("-[?/?]-"), lastLogs[2].split("-[?/?]-"), int(lastLogs[0][:-5][-1:]) #returns the untranslated list, translated list and the index the program left off on
    
    return [], [], -1 #Returns nothing if there was no log file of if the file did not have the expected data
 
#checks if a string is fully english
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

#Checks if string has CJK (chinese, japanese, korean) or alphanumic characters in it and returns true if it does
def hasCJKCharacters(s, **kwargs):
    if kwargs.get('mode', None) == "HiraKataOnly":
        return True if regex.search("([\p{Hiragana}\p{Katakana}])+", s) != None else False

    return True if regex.search("([\p{Hiragana}\p{Katakana}\p{Han}])+", s) != None else False
    
#Checks if there are chinese, korean or Japanese kanji (I.E. Hanzi) characters in the text - returns true if there isn't
def hasNoHanzi(s):
    return True if regex.search("(([\p{Hiragana}\p{Katakana}\p{Han}]*[\p{Hiragana}\p{Katakana}]+[\p{Hiragana}\p{Katakana}\p{Han}]*)|[a-zA-Z0-9]|[ａ-ｚＡ-Ｚ０-９])+", input) != None else False

#Checks if the text has hiragana or katakana, returns true is there isn't any
def hasNoHiraOrKata(s):
    #False if re.search("^(【)(.)*([\p{Hiragana}\p{Katakana}])+(.)*(】)$", input) != None else True
    return False if regex.search("([\p{Hiragana}\p{Katakana}])", s) != None else True

#Logs messages to text file
def logBehaviour(textToLog, logPath):
    if os.path.exists(logPath) and textToLog == "##Start Of Logs##\n#################\n":
        os.remove(logPath) 

    with open(logPath, 'a+', encoding='utf8') as file: 
        file.write(textToLog)
        file.close()

    return textToLog

#Handles 429 HTTP Reponse error by retrying and aborting if it fails again
def handle429Reponse(untranslatedList, translatedFileNames, fileName, logPath, translationNo):
    noOfTicks = 1 #12
    sleepLen = 1 #5
    print(
        "A HTTP 429 Error Occured. Too many translation requests were sent to the google API in a short period of time." +
        "Pausing script and resuming in about " + str(int((noOfTicks * sleepLen)/60)) + " minutes to attempt to rectify issue. \n> " + 
        "Note that if the script end with an error/exception again, the daily qouta of trasnlation the google API allows has likely been reached." + 
        "Note that none of these translations have been used to rename anything yet.\n" +
        "[", 
    end="")
    for i in range(noOfTicks):
        time.sleep(sleepLen)
        print("#", end="")
    print("]\n>> Attempting to Resume! <<\n> ", end="")

    rePattern = regex.compile("(【.*】)", regex.UNICODE)
    hasHiraAndKataRE = r"^【+((.*[\p{Hiragana}\p{Katakana}]+.*)|(.*([Dd][Rr][Aa][Mm][Aa]).*))】+"
    if regex.search(hasHiraAndKataRE, fileName) != None:
        fileName = rePattern.sub("", fileName)

    try:
        translation = translator.translate(fileName[:250], dest="en").text[:250]
        translatedFileNames.append(translation)
        print(logBehaviour("[" + str(translationNo) + "/" + str(len(untranslatedList)) + "]: " + fileName + "  \n\t\t===>  " + translation, logPath))
        translationNo += 1
    except:
        #Saves list of untranslated filename (in the order they were going to be translated) and the index that the program was on before siezing translation and ending
        logBehaviour("##Script ended abruptly! [Index = " + str(translationNo) + "]##" + "\n", logPath) #Log index and indication of abrupt end
        fileList = ""
        translatedFileList = ""

        #Log untranslated list
        for items in untranslatedList:
            fileList += str(items) + "-[?/?]-"
        logBehaviour(fileList.rsplit("-[?/?]-", 1)[0] + "\n", logPath)

        #Log translated file list
        for items in translatedFileNames:
            translatedFileList += str(items) + "-[?/?]-"
        logBehaviour(translatedFileList.rsplit("-[?/?]-", 1)[0], logPath)

        #Present used with error
        print("A HTTP 429 Error Occured Again! Aborting Script. Try running the script again, the scipt will resume from where it left off in this session.")

    return translatedFileNames

##########################
#Main function placeholder
##########################
runScript()

##Start Of Logs##
#################

> [1/943]: DRAMA CD vol5  
		===>  [[Same as before - no translation]]
> IN! - SERVANT x SERVICE x WORKING!! ドラマCD
OUT! - SERVANT x SERVICE x WORKING!! ドラマCD
[2/943]: SERVANT x SERVICE x WORKING!! ドラマCD  
		===>  SERVANT X SERVICE X Working !! Drama CD
> [3/943]: 【のうりん】 ドラマCD 【アニメ‘ｓ jp】  
		===>  【Uri】 Drama CD 【Anime 'S JP】
> [4/943]: 【ドラマＣＤ】【作業用】俺の彼女と幼なじみが修羅場すぎる 間島淳司 豊崎愛生 斉藤千和  
		===>  [Drama CD] [For work] My girlfriend and childhood friend are too shambles Mid Island Toyosaki Aisei Saito Saito
> [5/943]: 【ドラマＣＤ】【作業用】食戟のソーマ１１巻 同梱 松岡禎丞 種田利沙高橋未奈美  
		===>  【Drama CD】 【For work】 Soma Soma 11 volumes of food
> IN! - 【作業用BGM】中二病でも恋がしたいドラマCD集
OUT! - 中二病でも恋がしたいドラマCD集
[6/943]: 中二病でも恋がしたいドラマCD集  
		===>  Drama CD collection that I want to fall in love with you even if you have a second illness
> IN! - 【熟肉】龙王的工作！第十一卷广播剧Drama CD
OUT! - 龙王的工作！第十一卷广播剧Drama CD
[7/943]: 龙王的工作！第十一卷广播剧Drama CD  
		===>  Dragon King's work!Eleventh Volume Radio Drama Drama CD
> I